In [1]:
import pandas as pd

import apache_beam as beam
from apache_beam.io import WriteToBigQuery
from apache_beam.io.gcp.gcsio import GcsIO
from apache_beam.io.filebasedsource import FileBasedSource

import datetime

In [2]:
PROJECT_ID = 'assetinsure-surety-data-models'
BUCKET = 'gs://surety-data-models'
TABLE_SPEC = f'{PROJECT_ID}.ls_panthers_test.panters-test-table-1' # check if . or : between project ID and dataset

In [3]:
table_schema = {
        'fields': [
            {'name': 'ID', 'type': 'NUMERIC'},
            {'name': 'CompanyName', 'type': 'STRING'},
            {'name': 'Date', 'type': 'DATETIME'}
        ]
    }

In [69]:
new_df = pd.read_excel("Book.xlsm")
new_df.head()

,number,name,date,value,boolean
0,101,test,2023-08-02,200.45,False
1,102,Real Test,2023-01-05,3095452.85,False
2,103,Final,2021-06-09,287.11,True


In [70]:
new_df.dtypes

number              int64
name               object
date       datetime64[ns]
value             float64
boolean              bool
dtype: object

In [71]:
pd.to_datetime(df['col1'])

KeyError: 'col1'

In [72]:
new_df["timestamp"] = datetime.datetime.now()
new_df

,number,name,date,value,boolean,timestamp
0,101,test,2023-08-02,200.45,False,2023-09-20 18:49:59.834394
1,102,Real Test,2023-01-05,3095452.85,False,2023-09-20 18:49:59.834394
2,103,Final,2021-06-09,287.11,True,2023-09-20 18:49:59.834394


In [73]:
new_df.dtypes

number                int64
name                 object
date         datetime64[ns]
value               float64
boolean                bool
timestamp    datetime64[us]
dtype: object

In [25]:
file_name = f"{BUCKET}/input/Panthers Financial Model Oct-22.xlsm"
with GcsIO().open(file_name) as f:
            df = pd.read_excel(f)

In [5]:
df.head()

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 246,Unnamed: 247,Unnamed: 248,Unnamed: 249,Unnamed: 250,Unnamed: 251,Unnamed: 252,Unnamed: 253,Unnamed: 254,Unnamed: 255
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Penrith Rugby League Club,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,31 October 2020 (A$'000)),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Income Statement,0,NaN,NaN,NaN,Mgmt,NaN,Audited,NaN,Audited,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
df[df.iloc[:, 0].notnull()].index[0]

2

In [7]:
first_non_null_index = df[df.iloc[:, 0].notnull()].index[0]
first_non_null_index

2

In [8]:
company_name = df.iloc[first_non_null_index, 0]
company_name
print(company_name)

Penrith Rugby League Club


In [9]:
current_time = datetime.datetime.now()
current_time
print(current_time)

2023-09-20 13:29:11.872757


In [66]:
row_data = {
    'ID': 1,  # Replace with the actual ID if available
    'CompanyName': company_name,
    'Date': current_time
}
row_data

{'ID': 1,
 'CompanyName': 'Penrith Rugby League Club',
 'Date': datetime.datetime(2023, 9, 20, 13, 29, 11, 872757)}

In [67]:
test_df = pd.DataFrame.from_dict(row_data, orient='index').T
test_df

,ID,CompanyName,Date
0,1,Penrith Rugby League Club,2023-09-20 13:29:11.872757


In [68]:
for _, row in test_df.iterrows():
    print( row.to_dict())
    print(_)

{'ID': 1, 'CompanyName': 'Penrith Rugby League Club', 'Date': datetime.datetime(2023, 9, 20, 13, 29, 11, 872757)}
0


In [64]:
        row_data = {
            'ID': [2,3,4],  # Replace with the actual ID if available
            'CompanyName': [company_name,company_name,company_name],
            'Date': [current_time,current_time,current_time]
            }

In [65]:
test_df = pd.DataFrame.from_dict(row_data, orient='index').T
test_df

,ID,CompanyName,Date
0,2,Penrith Rugby League Club,2023-09-20 13:29:11.872757
1,3,Penrith Rugby League Club,2023-09-20 13:29:11.872757
2,4,Penrith Rugby League Club,2023-09-20 13:29:11.872757


In [40]:
test_df.dtypes.items()

In [63]:
bq_schema = pandas_dataframe_to_bq_schema(new_df)
print(bq_schema)

[{'name': 'number', 'type': 'INTEGER'}, {'name': 'name', 'type': 'STRING'}, {'name': 'date', 'type': 'TIMESTAMP'}, {'name': 'value', 'type': 'FLOAT'}, {'name': 'boolean', 'type': 'BOOLEAN'}]


In [41]:
import pandas as pd

def pandas_dataframe_to_bq_schema(df):
    """
    Create a BigQuery schema from a Pandas DataFrame.

    Parameters:
    df (pd.DataFrame): The DataFrame for which you want to generate a BigQuery schema.

    Returns:
    list: A list of dictionaries representing the BigQuery schema.
    """
    bq_schema = []

    for column_name, dtype in df.dtypes.items():
        field_type = None

        if dtype == 'int64':
            field_type = 'INTEGER'
        elif dtype == 'float64':
            field_type = 'FLOAT'
        elif dtype == 'bool':
            field_type = 'BOOLEAN'
        elif dtype == 'datetime64[ns]':
            field_type = 'TIMESTAMP'
        elif dtype == 'object':
            field_type = 'STRING'

        if field_type:
            bq_schema.append({'name': column_name, 'type': field_type})

    return bq_schema
